## interact with a locally hosted SLM (QWEN) to summarize web articles

In [1]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelInfo , UserMessage 
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

from dotenv import load_dotenv
load_dotenv()
import yaml

In [6]:
qwen_model = OpenAIChatCompletionClient(
        model="Community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf",
        base_url="http://localhost:8080/v1",
        model_info=ModelInfo(vision=False, function_calling=True, json_output=False, family="unknown"),
        api_key="lm-studio",
    ) 
# Setup server params for local filesystem access
fetch_mcp_server = StdioServerParams(command="uvx", args=["mcp-server-fetch"])
tools = await mcp_server_tools(fetch_mcp_server)

result = await qwen_model.create(messages=[UserMessage(source="user", content="Summarize the content of https://macleans.ca/society/canadas-new-nationalism/")], tools=tools)

print()
print(result.content)
print()



/Users/arturoquiroga/GITHUB/AZURE AI AGENT SERVICE/.venv/lib/python3.13/site-packages/autogen_ext/models/openai/_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)



The article "Canada's New Nationalism" by Paul Wells, published in Macleans.ca, discusses the concept of nationalism and its current relevance in Canada. Here's a summary:

Wells argues that modern Canadian politics have been shaped by a new type of nationalism that is distinct from traditional forms of nationalism. This "new nationalism" is characterized by a focus on national identity, cultural heritage, and economic sovereignty.

The article highlights three key aspects of this new nationalism:

1. **National Identity**: Canadians are redefining what it means to be Canadian, with a renewed emphasis on indigenous culture, diversity, and the country's natural resources.
2. **Cultural Heritage**: The debate over Canadian culture, including issues like Bill 21 (French language law in Quebec) and the 2019 federal election, showcases the importance of cultural identity and heritage in shaping national policy.
3. **Economic Sovereignty**: The rise of anti-globalization sentiment and conce

/var/folders/dj/qp0fwj152ks28q9cn0_rd3fw0000gn/T/ipykernel_8977/1977092416.py:11: UserWarning: Resolved model mismatch: Community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf != LM Studio Community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to LM Studio Community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf to enhance token/cost estimation and suppress this warning.
  result = await qwen_model.create(messages=[UserMessage(source="user", content="Summarize the content of https://macleans.ca/society/canadas-new-nationalism/")], tools=tools)


## Additional interactions with a local SLM

In [3]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:8080/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="Meta-Llama-3-8B-Instruct-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message.content)

I'm a bot so fine,
My name is LLaMA, nice to be mine.
I'll chat with you all day long,
Answering questions right and strong.
I'm here to help and have some fun,
Let's talk in rhymes, the day has just begun!


In [4]:
# Chat with an intelligent assistant in your terminal
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:8080/v1", api_key="lm-studio")

history = [
    {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."},
    {"role": "user", "content": "Hello, introduce yourself to someone opening this program for the first time. Be concise."},
]

while True:
    completion = client.chat.completions.create(
        model="Mungert/Qwen2.5-7B-Instruct-1M-GGUF",
        messages=history,
        temperature=0.7,
        stream=True,
    )

    new_message = {"role": "assistant", "content": ""}
    
    for chunk in completion:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
            new_message["content"] += chunk.choices[0].delta.content

    history.append(new_message)
    
    # Uncomment to see chat history
    import json
    yellow_color = "\033[33m"  # Yellow ANSI code
    reset_color = "\033[0m"
    print(f"{yellow_color}\n{'-'*20} History dump {'-'*20}\n")
    print(json.dumps(history, indent=2))
    print(f"\n{'-'*55}\n{reset_color}")

    print()
    history.append({"role": "user", "content": input("> ")})

Hi there! I'm Language Model, your friendly AI companion. I'll help you with any questions or tasks you have, from answering curiosity-driven queries to providing guidance on a wide range of topics. Let's chat and explore the possibilities together!
-------------------- History dump --------------------

[
  {
    "role": "system",
    "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."
  },
  {
    "role": "user",
    "content": "Hello, introduce yourself to someone opening this program for the first time. Be concise."
  },
  {
    "role": "assistant",
    "content": "Hi there! I'm Language Model, your friendly AI companion. I'll help you with any questions or tasks you have, from answering curiosity-driven queries to providing guidance on a wide range of topics. Let's chat and explore the possibilities together!"
  }
]

-------------------------------------------------------


The Laplace transform is a powerful m

BadRequestError: Error code: 400 - {'error': "'messages' array must only contain objects with a 'content' field that is not empty."}